<a href="https://colab.research.google.com/github/ccmkaaa/colab/blob/main/dz/dz_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import time
import regex as re
from bs4 import BeautifulSoup
from tabulate import tabulate

In [ ]:
keywords = ['python', 'анализ данных']

def get_posts_from_habr(pages, keyword):                                                  # we need keywords & pages to output
  url = 'https://habr.com/ru/search/'
  params ={                                                                                # params 4 req
      'q': keyword,
      'order': 'relevance',
      'target_type': 'posts'
  }
  headers ={                                                                               # headers 4 req
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
  }

  habr_posts = pd.DataFrame()                                                               # empty df
  for page in range(1, pages+1):                                                            # "перелистывание" страниц
    params['page'] = page                                                                   # input page param 4 req
    url = f'https://habr.com/ru/search/page{page}/'                                         # url for requets
    response = requests.get(url, params=params, headers=headers)                            # вызов GET
    if response.status_code == 404:                             
      continue
    time.sleep(0.33) 
    soup = BeautifulSoup(response.text)                                                     # parse to BS

    posts = soup.find_all('article', class_='tm-articles-list__item')                       # search for posts
    for post in posts:                                                                      # loop for each post
      row = get_row_from_post(post)                                                         # get row to df
      habr_posts = pd.concat([habr_posts, pd.DataFrame([row])])                             # concat row

  return habr_posts.reset_index(drop=True)                                                  # return df

def get_row_from_post(post):                                                                                          # есть два вида текста под постом, их тип никак не связан с видом поста (мегапост или обычный)
  if post.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1'):                  # 1 type 
    text = post.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1').text        # get text
  elif post.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2'):                # 2 type 
    text = post.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2').text        # get text
  else: text = None                                                                                                              # else None

  if post.find('div', class_='tm-megapost-snippet'):                                                                # мегапост
    title = post.find('h2', class_='tm-megapost-snippet__title').text                                                            # get title
    tags = ';'.join([tag.text for tag in post.find_all('li', class_='tm-megapost-snippet__hub')])                                # get tags
    tags = clear_tags(tags)                                                                                                      # clear tags
    link = post.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__card').get('href')                              # get link
    date = post.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__date').find('time').get('datetime')             # get date
  else:                                                                                                             # обычный пост
    tags = ';'.join([tag.text for tag in post.find_all('a', class_='tm-article-snippet__hubs-item-link')])                       # get tags
    tags = clear_tags(tags)                                                                                                      # clear tags
    title = post.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').text                                # get title
    link = post.find('a', class_='tm-article-snippet__title-link').get('href')                                                   # get link
    date = post.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime')                       # get date

  if post.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating'):                      # if (likes > 0) & (likes < 0) 
    likes = int(post.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text)      # get likes   
  else: likes = 0                                                                                                                                                                                       # else likes = 0

  view = float(post.find('span', class_='tm-icon-counter__value').text.replace('K', ''))                                         # get view
  bookmarks = int(post.find('span', class_="bookmarks-button__counter").text)                                                    # get bookmarks
  comments = int(post.find('span', class_='tm-article-comments-counter-link__value').text)                                       # get comments

  row = {'datetime': date, 'title': title, 'link': link, 'tags': tags, 'like': likes, 'view(in k)': view, 'bookmarks': bookmarks, 'comments': comments, 'text': text}                # get row 
  return row                                                                                                    # return row
 
def clear_tags(tags):                                                                                       # func 4 remove '*' char or ' ;' to ';'
  tags = ''.join([re.sub('[*]', '', tag) for tag in tags])                                                 
  return re.sub(r' ;', ';', tags)

def get_posts_4_every_keyword(keywords):
  for keyword in keywords:         
    df = get_posts_from_habr(4, keyword)                                     # call (4 pages and 'keyword)
    print(tabulate(df, headers='keys', tablefmt='psql'))
    print('#####')

get_posts_4_every_keyword(keywords)

+----+--------------------------+---------------------------------------------------------------------------------------------------------------+---------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+--------+--------------+-------------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------